## Ratings tracker
the code below is for appsscript in googlecloud

function createFormWithNames() {
  // List of 12 names
  const names = [
    "Player 1", "Player 2", "Player 3", "Player 4", "Player 5", "Player 6",
    "Player 7", "Player 8", "Player 9", "Player 10", "Player 11", "Player 12"
  ];

  // Get the active form
  const form = FormApp.getActiveForm();

  // Set the form title and description
  form.setTitle("Player Rating Form");
  form.setDescription("Please rate each player on a scale of 1 to 10.");

  // Add a rating question for each name
  names.forEach((name, index) => {
    form.addScaleItem()
      .setTitle(`Rate ${name}`)
      .setBounds(1, 10) // Set the scale from 1 to 10
      .setRequired(true); // Make the question required
  });

  // Notify the user that the form has been created
  Logger.log("Form created successfully!");
}

In [1]:
# Create datasets for batsmen and bowlers

In [ ]:
# Bowlers

In [ ]:
# Main tracker
Bowling_Stats_Main = {}
# Match tracker
Match = {}


# Bowler deets
players = {}
# Figures
figs = []

In [3]:
Bowlers = {'Match1':
           {'Jadeja':[],'Noor':[]}
          }

In [ ]:
Jadeja = {}

In [ ]:
Match1 = [wickets,balls,runs]
matchinfo = {'overs': 4, 'runs': 25, 'wickets': 2, 'dots': 10, 'maidens': 1}

In [4]:
# Batsmen

In [6]:
matchinfobat = {'overs': 4, 'runs': 25,'4s':2,'6s':2,'OUT':0}

In [8]:
batsmen = {'Match1':
          {'Conway':matchinfobat,
          'Gaikwad':matchinfobat}
          }

In [41]:
batsmen['Match1']['Conway'].keys()

dict_keys(['overs', 'runs', '4s', '6s', 'OUT'])

In [51]:
# Go through all Batsmen
for i in batsmen:
    # Go thorugh indivudual names
    for x in batsmen[i]:
        print(x)
        print("_"*15)
        # Go through Ind stats
        for stats in batsmen[i][x]:
            print(f" {stats} | {batsmen[i][x][stats]}")


Conway
_______________
 overs | 4
 runs | 25
 4s | 2
 6s | 2
 OUT | 0
Gaikwad
_______________
 overs | 4
 runs | 25
 4s | 2
 6s | 2
 OUT | 0


In [34]:
for i in batsmen[i]:
    print(i)


Conway
Gaikwad


In [23]:
batsmen.items()

dict_items([('Match1', {'Conway': {'overs': 4, 'runs': 25, '4s': 2, '6s': 2, 'OUT': 0}, 'Gaikwad': {'overs': 4, 'runs': 25, '4s': 2, '6s': 2, 'OUT': 0}})])

In [52]:
# Webscraping

In [74]:
import pandas as pd
import re
from pathlib import Path

def convert_google_sheet_url(url):
    # Regular expression to match and capture the necessary part of the URL
    pattern = r'https://docs\.google\.com/spreadsheets/d/([a-zA-Z0-9-_]+)(/edit#gid=(\d+)|/edit.*)?'

    # Replace function to construct the new URL for CSV export
    # If gid is present in the URL, it includes it in the export URL, otherwise, it's omitted
    replacement = lambda m: f'https://docs.google.com/spreadsheets/d/{m.group(1)}/export?' + (f'gid={m.group(3)}&' if m.group(3) else '') + 'format=csv'

    # Replace using regex
    new_url = re.sub(pattern, replacement, url)

    # Add it to a dataset
    data = df = pd.read_csv(new_url)
    return data

data = convert_google_sheet_url("https://docs.google.com/spreadsheets/d/1-V1vdP9zblKaU97gUUsREnIBiGjaTSNmNiD4p5OjNJ0/edit?gid=333621817#gid=333621817")

In [94]:
averages = data.iloc[:,1:].mean()  # Returns average for each player (column)
match1_ratings = compile(data.iloc[:,1:])

Noor Ahmed : 9.71
Rutraj Gaikwad: 9.57
Rachin Ravindra : 8.81
Khaleel Ahmed : 8.67
MS Dhoni: 7.38
Ravichandra Ashwin : 7.29
Ravindra Jadeja: 6.67
Nathan Ellis: 6.05
Shivam Dube: 3.86
Sam Curran: 2.76
Rahul Tripati: 2.57
Deepak Hooda: 2.43
__________________________________________________
🏆 Top 3:
__________________________________________________
Noor Ahmed : 9.71
Rutraj Gaikwad: 9.57
Rachin Ravindra : 8.81
__________________________________________________

⚠️ Needs Improvement:
__________________________________________________
Deepak Hooda: 2.43
Rahul Tripati: 2.57
Sam Curran: 2.76


In [93]:
def compile(D):
    averages = D.mean()  # Returns average for each player (column)
    sorted = averages.sort_values(ascending=False)
    top_3 = averages.nlargest(3)
    bottom_3 = averages.nsmallest(3)

     
    for player, rating in sorted.items():
        print(f"{player}: {rating:.2f}")
       
    print("_"*50)
    print("🏆 Top 3:")
    print("_"*50)
    for player, rating in top_3.items():
        print(f"{player}: {rating:.2f}")

    print("_"*50)
    print("\n⚠️ Needs Improvement:")
    print("_"*50)
    for player, rating in bottom_3.items():
        print(f"{player}: {rating:.2f}")
    return sorted

In [102]:
def update_csv(Title,Ratings):
    # Create or update CSV
    csv_path = "Compiled_ratings.csv"
    if Path(csv_path).exists():
        # Load existing data
        df = pd.read_csv(csv_path)
    else:
        # Create new DataFrame
        df = pd.DataFrame()
    
    # Add/update match data
    df[f'{Title}'] = pd.Series({Ratings})
    
    # Save to CSV
    df.to_csv(csv_path)

In [104]:
def update_csv(Title, Ratings):
    """
    Update CSV with new match ratings
    Args:
        Title: str - Match identifier (e.g., 'Delhi', 'Bangalore')
        Ratings: dict - {player_name: rating} or pandas Series
    """
    csv_path = "Compiled_ratings.csv"
    
    # Convert Ratings to Series if it's a dict
    if isinstance(Ratings, dict):
        Ratings = pd.Series(Ratings)
    
    # Ensure Ratings is a Series with player names as index
    Ratings = Ratings.rename(Title).round(2)
    
    if Path(csv_path).exists():
        # Load existing data with player names as index
        df = pd.read_csv(csv_path, index_col=0)
        
        # Merge new ratings
        df = df.join(Ratings, how='outer')
    else:
        # Create new DataFrame from ratings
        df = pd.DataFrame(Ratings)
    
    # Save to CSV
    df.to_csv(csv_path)
    return df

In [105]:
update_csv("Mumbai",match1_ratings.items())

AttributeError: 'zip' object has no attribute 'rename'

In [98]:
for i in match1_ratings.items():
    print(i)

('Noor Ahmed ', 9.714285714285714)
('Rutraj Gaikwad', 9.571428571428571)
('Rachin Ravindra ', 8.80952380952381)
('Khaleel Ahmed ', 8.666666666666666)
('MS Dhoni', 7.380952380952381)
('Ravichandra Ashwin ', 7.285714285714286)
('Ravindra Jadeja', 6.666666666666667)
('Nathan Ellis', 6.0476190476190474)
('Shivam Dube', 3.857142857142857)
('Sam Curran', 2.761904761904762)
('Rahul Tripati', 2.5714285714285716)
('Deepak Hooda', 2.4285714285714284)


In [101]:
type(match1_ratings)

pandas.core.series.Series

In [60]:
sorted[3:-3]

Khaleel Ahmed          8.666667
MS Dhoni               7.380952
Ravichandra Ashwin     7.285714
Ravindra Jadeja        6.666667
Nathan Ellis           6.047619
Shivam Dube            3.857143
dtype: float64